In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time 
import pandas as pd

# Initialize the webdriver
# de manière basique
# driver = webdriver.Chrome()

# avec ChromeDriverManager mieux car update du driver automatiquement
driver = webdriver.Chrome(ChromeDriverManager().install())

# si on a télécharger le driver directement dans le dossier 
# service = Service(executable_path="./chromedriver.exe")
# driver = webdriver.Chrome(service=service)

C:\Users\Camille\AppData\Local\Temp\ipykernel_21528\66893964.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
# Navigate to the website
# driver.get("https://www.billboard.com/charts/hot-100/")
driver.get("https://www.billboard.com/charts/billboard-global-200/")

# Wait for the page to load
driver.implicitly_wait(10)

# reject all cookies
driver.find_element(By.XPATH,"/html/body/div[6]/div[2]/div/div/div[2]/div/div/button[1]").click()

In [4]:
iterateur = list(range(1,110,11))
iterateur.remove(1)
iterateur200 = list(range(1,220,11))
iterateur200.remove(1)

In [5]:
def scraper(list,xpath,iterateur,nb_max):
    for i in range(2,nb_max):
        if i not in iterateur : 
            WebElement = driver.find_element(By.XPATH,xpath%(i))
            list.append(WebElement.text)

    return list

In [6]:
XpathTitle = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/h3"
XpathArtist = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/span"
XpathRank = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[1]/span "
XpathLastWeek = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[4]/span"
XpathPeakPosition = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[5]/span"
XpathWeeksOnChart = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[6]/span"

In [7]:
allXpath = [XpathTitle,XpathArtist,XpathRank,XpathLastWeek,XpathPeakPosition,XpathWeeksOnChart]

In [8]:
title,artist,rank,last_week,peak_pos,weeks_on_chart = [],[],[],[],[],[]
allList = [title,artist,rank,last_week,peak_pos,weeks_on_chart]

In [9]:
# for i, j in zip(allList, allXpath):
#     i = scraper(i,j,iterateur200,221)

In [10]:
i = [scraper(i,j,iterateur200,221) for i, j in zip(allList, allXpath)]

In [11]:
# title = scraper(title,XpathTitle,iterateur200,221)
# artist = scraper(artist,XpathArtist,iterateur200,221)
# rank = scraper(rank,XpathRank,iterateur200,221)
# last_week = scraper(last_week,XpathLastWeek,iterateur200,221)
# peak_pos = scraper(peak_pos,XpathPeakPosition,iterateur200,221)
# weeks_on_chart = scraper(weeks_on_chart,XpathWeeksOnChart,iterateur200,221)

In [13]:
df = pd.DataFrame(list(zip(title,artist,rank,last_week,peak_pos,weeks_on_chart)),columns=['Title','Artist','Rank','Last Week','Peak Positon','Weeks on charts'])
df

,Title,Artist,Rank,Last Week,Peak Positon,Weeks on charts
0,Flowers,Miley Cyrus,1,1,1,2
1,"Bzrp Music Sessions, Vol. 53",Bizarrap & Shakira,2,2,2,3
2,Kill Bill,SZA,3,3,1,7
3,Calm Down,Rema & Selena Gomez,4,4,3,30
4,Unholy,Sam Smith & Kim Petras,5,6,1,18
...,...,...,...,...,...,...
195,Wasted On You,Morgan Wallen,196,-,19,49
196,Gatita,Bellakath,197,147,113,6
197,Wake Me Up!,Avicii,198,200,154,34
198,Only Love Can Hurt Like This,Paloma Faith,199,-,75,26
